<a href="https://colab.research.google.com/github/tunhu-da/Credit-Card-Fraud-Detection/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

In [3]:
!pip install -U imbalanced-learn  # Update imblearn library

In [39]:
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from matplotlib import pyplot as plt
plt.style.use('ggplot')
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score, precision_score, recall_score
from imblearn.over_sampling import RandomOverSampler
import warnings

warnings.filterwarnings('ignore')

In [3]:
os.environ['KAGGLE_CONFIG_DIR'] = "/content/.kaggle"
!mkdir -p /content/.kaggle
!cp kaggle.json /content/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [4]:
!kaggle datasets download -d mlg-ulb/creditcardfraud

Dataset URL: https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud
License(s): DbCL-1.0
 85% 56.0M/66.0M [00:00<00:00, 127MB/s] 
100% 66.0M/66.0M [00:00<00:00, 114MB/s]


In [5]:
!unzip /content/creditcardfraud.zip -d /content

Archive:  /content/creditcardfraud.zip
  inflating: /content/creditcard.csv  


In [6]:
df = pd.read_csv('/content/creditcard.csv')

In [7]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [8]:
pd.options.display.max_columns = None

In [9]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [10]:
df.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,4.356170,-1.593105,2.711941,-0.689256,4.626942,-0.924459,1.107641,1.991691,0.510632,-0.682920,1.475829,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,-0.975926,-0.150189,0.915802,1.214756,-0.675143,1.164931,-0.711757,-0.025693,-1.221179,-1.545556,0.059616,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,-0.484782,0.411614,0.063119,-0.183699,-0.510602,1.329284,0.140716,0.313502,0.395652,-0.577252,0.001396,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,-0.399126,-1.933849,-0.962886,-1.042082,0.449624,1.962563,-0.608577,0.509928,1.113981,2.897849,0.127434,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0
284806,172792.0,-0.533413,-0.189733,0.703337,-0.506271,-0.012546,-0.649617,1.577006,-0.414650,0.486180,-0.915427,-1.040458,-0.031513,-0.188093,-0.084316,0.041333,-0.302620,-0.660377,0.167430,-0.256117,0.382948,0.261057,0.643078,0.376777,0.008797,-0.473649,-0.818267,-0.002415,0.013649,217.00,0


In [11]:
df.shape

(284807, 31)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [13]:
df.isnull().sum()

,0
Time,0
V1,0
V2,0
V3,0
V4,0
V5,0
V6,0
V7,0
V8,0
V9,0


In [14]:
sc = StandardScaler()
df['Amount'] = sc.fit_transform(pd.DataFrame(df['Amount']))

In [15]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0.244964,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,-0.342475,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,1.160686,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0.140534,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,-0.073403,0


In [16]:
df.drop(['Time'], axis=1, inplace=True)

In [17]:
df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0.244964,0
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,-0.342475,0
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,1.160686,0
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0.140534,0
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,-0.073403,0


In [18]:
df.duplicated().any()

True

In [19]:
df = df.drop_duplicates()

In [20]:
df.shape

(275663, 30)

In [21]:
df['Class'].value_counts()

,count
Class,
0,275190
1,473


imbalanced -> oversampling / *undersampling*



In [23]:
X = df.drop(['Class'], axis=1)
y = df['Class']

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [44]:
X_train[:5]

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
273481,-0.161724,0.795576,-0.263689,-0.971826,0.729349,-0.287863,0.731175,0.134851,0.279731,0.130830,-0.479698,-0.632147,-2.023348,0.600722,-0.779528,0.480546,-0.917502,0.323813,0.574806,-0.009277,-0.332534,-0.843464,-0.037750,-1.105276,-0.405012,0.196066,0.339190,0.129651,-0.331840
74338,-0.949530,0.450853,-0.057063,-2.439902,2.300471,3.149770,0.067733,0.949800,0.054195,-0.368540,-0.236344,-0.221798,-0.063604,0.027274,0.729429,0.920491,-1.186138,0.093470,-0.344013,0.318038,-0.161192,-0.526617,-0.039507,1.033637,-0.116680,0.739483,0.362217,0.260821,-0.193346
256081,1.960680,-0.419682,-0.199108,0.465864,-0.705643,-0.292169,-0.675901,0.006359,1.142480,-0.025475,-0.536230,0.752957,0.657522,-0.359671,0.350844,0.155202,-0.351581,-0.244362,-0.466256,-0.140345,0.192411,0.768983,0.242880,1.159461,-0.306905,0.513329,-0.017098,-0.035060,-0.307251
329424,-13.897206,6.344280,-14.281666,5.581009,-12.887133,-3.146176,-15.450467,9.060281,-5.486121,-14.676470,5.148352,-10.749592,1.585489,-9.798012,0.733277,-12.675269,-20.740664,-8.153668,5.228342,-1.025228,3.058082,0.941180,-0.232710,0.763508,0.075456,-0.453840,-1.508968,-0.686836,-0.313289
141129,-0.269992,0.774448,-0.806605,-0.536963,0.978857,-0.708773,1.583865,-0.189871,-0.273690,-0.996162,-0.975583,-0.582573,-0.481428,-0.884695,-0.293266,0.212315,0.490163,0.048149,-0.126559,0.288760,-0.137241,-0.464638,0.135441,0.373024,0.020467,0.356541,0.191284,0.132617,0.123103


In [30]:
classifier = {
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier()
}

for name, clf in classifier.items():
  print(f'\n======={name}===========')
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)

  accuracy = accuracy_score(y_test, y_pred)
  print(f'Accuracy Score: {accuracy}')

  print(f'\nConfusion Matrix:\n {confusion_matrix(y_test, y_pred)}\n')
  print(classification_report(y_test, y_pred))


=======Logistic Regression===========
Accuracy Score: 0.9992563437505668

Confusion Matrix:
 [[55035     7]
 [   34    57]]

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     55042
           1       0.89      0.63      0.74        91

    accuracy                           1.00     55133
   macro avg       0.95      0.81      0.87     55133
weighted avg       1.00      1.00      1.00     55133


=======Decision Tree===========
Accuracy Score: 0.9989842743910181

Confusion Matrix:
 [[55010    32]
 [   24    67]]

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     55042
           1       0.68      0.74      0.71        91

    accuracy                           1.00     55133
   macro avg       0.84      0.87      0.85     55133
weighted avg       1.00      1.00      1.00     55133


=======Random Forest===========
Accuracy Score: 0.9994739992382058

Confusion Matrix:
 [[55037     

In [46]:
X = X.sample(frac=1, random_state=42)
y = y.sample(frac=1, random_state=42)

ros = RandomOverSampler()
X_ros, y_ros = ros.fit_resample(X, y)

In [47]:
X_ros.shape

(550380, 29)

In [48]:
y_ros.shape

(550380,)

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X_ros, y_ros, test_size = 0.2, random_state = 42)

In [50]:
X_train[:5]

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
273481,1.047498,-0.040982,1.465944,1.907840,-0.936683,0.149287,-0.629575,0.212520,0.074706,0.364597,-0.222791,0.216613,-0.030194,-0.271622,0.445707,0.476300,-0.046132,-0.852869,-1.175766,-0.088166,-0.014733,-0.005272,0.135394,0.435286,-0.063161,0.889902,-0.022938,0.027230,-0.221093
74338,-0.252913,0.739525,-1.491578,-1.565861,0.761827,-1.285801,1.360073,-0.416580,-1.709309,0.561089,-1.471688,-0.776284,-0.435738,0.720699,-0.861257,-2.221567,0.067528,0.861286,-0.841762,-0.697444,0.055185,0.666240,-0.218662,0.794982,-0.407734,0.751106,-0.428439,-0.078688,-0.221293
256081,1.207441,0.287844,0.741376,0.746258,-0.687549,-1.278888,0.130465,-0.280638,-0.095715,-0.090337,0.377202,0.908221,0.905228,0.137395,0.919082,0.116474,-0.268016,-0.798195,-0.326981,-0.050163,-0.209603,-0.594748,0.186944,0.931747,0.166413,0.058397,-0.025447,0.027313,-0.313328
329424,-21.885434,12.930505,-24.098872,6.203314,-16.466099,-4.459842,-16.519836,14.535565,-3.897022,-8.650758,5.375434,-7.567214,0.461808,-7.827891,-0.022815,-6.578042,-13.002459,-4.884467,1.104355,1.611998,1.762232,-1.579055,-0.951043,0.134565,1.507110,-0.222671,1.527655,0.453699,0.046539
141129,1.043656,-1.663870,-0.403771,-1.333184,-1.266211,-0.725187,-0.293248,-0.244382,-2.361917,1.526208,0.924698,-0.552320,-0.183085,0.306178,-0.282172,-0.304173,0.324511,0.163042,0.370080,0.151423,-0.331375,-1.246144,-0.112680,-0.049060,0.242123,-0.471612,-0.052239,0.044329,0.701985


In [52]:
classifier = {
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier()
}

for name, clf in classifier.items():
  print(f'\n======={name}===========')
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)

  accuracy = accuracy_score(y_test, y_pred)
  print(f'Accuracy Score: {accuracy}')

  print(f'Confusion Matrix: \n{confusion_matrix(y_test, y_pred)}')
  print(classification_report(y_test, y_pred))


=======Logistic Regression===========
Accuracy Score: 0.9483992877648171
Confusion Matrix: 
[[53846  1237]
 [ 4443 50550]]
              precision    recall  f1-score   support

           0       0.92      0.98      0.95     55083
           1       0.98      0.92      0.95     54993

    accuracy                           0.95    110076
   macro avg       0.95      0.95      0.95    110076
weighted avg       0.95      0.95      0.95    110076


=======Decision Tree===========
Accuracy Score: 0.9997637995566699
Confusion Matrix: 
[[55057    26]
 [    0 54993]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     55083
           1       1.00      1.00      1.00     54993

    accuracy                           1.00    110076
   macro avg       1.00      1.00      1.00    110076
weighted avg       1.00      1.00      1.00    110076


=======Random Forest===========
Accuracy Score: 0.9999545768378212
Confusion Matrix: 
[[55078     5]
 [